<a href="https://colab.research.google.com/github/iypc-team/CoLab/blob/master/Access_Google_Drive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# %%writefile AccessGoogleDrive.py
# 10/29/2020
from __future__ import absolute_import, print_function
global agd
agd = None
class AccessGoogleDrive:
    '''Import PyDrive and associated libraries.
    This only needs to be done once per notebook.
    '''
    # from pydrive.auth import GoogleAuth
    # from pydrive.drive import GoogleDrive
    # from google.colab import auth
    # from oauth2client.client import GoogleCredentials

    # auth.authenticate_user()
    # gauth = GoogleAuth()
    # gauth.credentials = GoogleCredentials.get_application_default()
    # drive = GoogleDrive(gauth)

    # import glob, os
    # from os.path import basename, join, realpath
    # from pathlib import Path

    def __init__(self, agd):
        from pydrive.auth import GoogleAuth
        from pydrive.drive import GoogleDrive
        from google.colab import auth
        from oauth2client.client import GoogleCredentials

        auth.authenticate_user()
        self.gauth = GoogleAuth()
        self.gauth.credentials = GoogleCredentials.get_application_default() 
        self.drive = GoogleDrive(self.gauth)

        self.agd = agd
        self.contentPath = '/content/'
        self.drivePath = '/content/drive/'
        self.myDrivePath = '/content/drive/My Drive'
        self.fileIdDictionary={}
        self.fileTypesDict={}
        self.directoryListDict={}
        self.allFilesList=[]
        self.directoryList=[]
        self.fileList=[]
        self.fil = None
        self.C = None

    # auth.authenticate_user()
    # gauth = GoogleAuth()
    # gauth.credentials = GoogleCredentials.get_application_default()
    # drive = GoogleDrive(gauth)
    # self.gdrive = None

    def mountGoogleDrive(self):
        '''ToDo...
        fix help
        '''
        import glob, os
        from os.path import abspath
        from pathlib import Path
        from google.colab import drive as gdr
        self.gdrive = gdr.mount('/content/drive', force_remount=True)
        self.allFilesList = glob.glob('**', recursive=True)
        print(f'{len(self.allFilesList)} files in Google Drive')
        os.chdir('/content/drive/My Drive')
        from BashColors import C
        self.C = C
        print(f'cwd: {C.Green}{Path.cwd()}{C.ColorOff}\n')
    
    def listGoogleDriveFiles(self, silent=True):
        '''List all files in the root
        returns fileIdDictionary
        Search query reference:
        https://developers.google.com/drive/v2/web/search-parameters'''
        
        import BashColors
        from BashColors import C
        from os.path import abspath
        fileTypesDict = {}
        list = self.drive.ListFile({'q': "title contains '.*' and 'root' in parents"}).GetList() 
        self.fileList = list.copy()
        counter=0
        for fil in self.fileList:
            fileName = fil['title']
            file_id = fil['id']
            fileId = fil['id']
            fileMimeType = fil['mimeType']
            labels = fil['labels'] # Returns a dictionary
            fullPath = abspath(fileName)
            counter+=1
            self.fileIdDictionary[counter]=(fileName, fil['id'], fullPath)

            if fileMimeType=='application/vnd.google-apps.folder':
                self.directoryList.append(fullPath)
                self.directoryListDict[fileName]=(fil['id'], fullPath)

            if not silent:
                if fileMimeType=='application/vnd.google-apps.folder' \
                and not fullPath.__contains__('__pycache__'):
                    print(f'{self.C.Blue}{fileName}')
                    print(f'{fileMimeType}')
                    print(f'{C.White}id: {file_id}')
                    for key in labels.keys():
                        print(f'{key}: {labels[key]}')

                elif not fullPath.__contains__('__pycache__'):
                    print(f'{C.White}{counter}. {fileName}')
                    print(f'{fileMimeType}')
                    print(f'id: {file_id}')
                    for key in labels.keys():
                        print(f'{key}: {labels[key]}')
                print()

agd = AccessGoogleDrive(agd=agd)
# agd.listGoogleDriveFiles(silent=False)

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
os.chdir('/content/drive/My Drive')

In [ ]:

import AccessGoogleDrive
def selectFile(itemNumber:int):
    '''Returns selected file id number'''
    counter=0
    for item in fileIdDictionary.keys():
        counter+=1
        dict_item = fileIdDictionary[item]
        print(counter, dict_item[0])

    selection = fileIdDictionary[itemNumber]
    selectName = selection[0]
    selectId = selection[1]
    print(f'{selectName}')
    return selectId


idd=selectFile(itemNumber=1)
print(idd)

# Saving data to Google Drive

In [ ]:

def createAndUploadFile(named:str, inDir:str, content:str):
    '''Create & upload a text file.'''
    currentPath=Path.cwd()
    dir_path=join('/content/drive/My Drive', inDir)
    if exists(dir_path):
        os.chdir(dir_path)
        print(Path.cwd)
    else:
        os.mkdir(dir_path)
        os.chdir(dir_path)
        print(Path.cwd())

    uploaded = drive.CreateFile({'title': named})
    uploaded.SetContentString(content)
    # uploaded.Upload()
    listGoogleDriveFiles(silent=False)
    print('Uploaded file with ID {}'.format(uploaded.get('id')))


In [ ]:
file_name='bullshit-2.txt'
fileContent='New content here.'
createAndUploadFile(named=file_name, inDir='testDir', content=fileContent)

In [ ]:
def downloadGoogleDriveFile(thisFileId):
    '''Download a file based on its file id'''
    print(thisFileId)
    file_id = thisFileId
    downloaded = drive.CreateFile({'id': file_id})
    print('Downloaded content "{}"'.format(downloaded.GetContentString()))

In [ ]:
downloadFile = selectFile(13)
downloadGoogleDriveFile(downloadFile)